## Test accuracy on 20 CIFAR-100 classes using EfficieentNetB0

### Import all the necessary libraries

In [9]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar100
import numpy as np

###  Load CIFAR-100 fine labels (100-class)

In [10]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

### Choose 20 classes

In [11]:
selected_classes = list(range(20))
train_mask = np.isin(y_train, selected_classes).flatten()
test_mask = np.isin(y_test, selected_classes).flatten()

x_train_20 = x_train[train_mask]
y_train_20 = y_train[train_mask]
x_test_20 = x_test[test_mask]
y_test_20 = y_test[test_mask]

### Map label to 0 - 19

In [12]:
label_map = {cls: i for i, cls in enumerate(selected_classes)}
y_train_20 = np.array([label_map[y[0]] for y in y_train_20])
y_test_20 = np.array([label_map[y[0]] for y in y_test_20])

### Resize to 128x128

In [13]:
x_train_20 = tf.image.resize(x_train_20, (128, 128)).numpy()
x_test_20 = tf.image.resize(x_test_20, (128, 128)).numpy()

2025-05-24 22:17:01.482020: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1966080000 exceeds 10% of free system memory.
2025-05-24 22:17:08.800013: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 393216000 exceeds 10% of free system memory.


### Preprocess

In [14]:
x_train_20 = preprocess_input(x_train_20)
x_test_20 = preprocess_input(x_test_20)

### Converts Label

In [15]:
y_train_20_cat = tf.keras.utils.to_categorical(y_train_20, 20)
y_test_20_cat = tf.keras.utils.to_categorical(y_test_20, 20)

### Load EfficieentNetB0 base model

In [ ]:
base_model = EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # freeze base model

# 8. Build the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(20, activation='softmax')
])

# 9. Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 10. Train
model.fit(x_train_20, y_train_20_cat, epochs=5, batch_size=64, validation_split=0.1)

# 11. Evaluate
loss, acc = model.evaluate(x_test_20, y_test_20_cat)
print("Test accuracy on 20 CIFAR-100 classes using EfficientNetV2B0:", acc)

Epoch 1/10
141/141 [==============================] - 47s 295ms/step - loss: 0.8584 - accuracy: 0.7547 - val_loss: 0.4874 - val_accuracy: 0.8440
Epoch 2/10
141/141 [==============================] - 40s 286ms/step - loss: 0.4074 - accuracy: 0.8676 - val_loss: 0.4494 - val_accuracy: 0.8590
Epoch 3/10
141/141 [==============================] - 40s 287ms/step - loss: 0.3188 - accuracy: 0.8962 - val_loss: 0.4441 - val_accuracy: 0.8540
Epoch 4/10
141/141 [==============================] - 40s 285ms/step - loss: 0.2682 - accuracy: 0.9076 - val_loss: 0.4223 - val_accuracy: 0.8650
Epoch 5/10
141/141 [==============================] - 40s 285ms/step - loss: 0.2198 - accuracy: 0.9287 - val_loss: 0.4467 - val_accuracy: 0.8620
Epoch 6/10
141/141 [==============================] - 40s 284ms/step - loss: 0.1871 - accuracy: 0.9371 - val_loss: 0.4311 - val_accuracy: 0.8640
Epoch 7/10
141/141 [==============================] - 40s 286ms/step - loss: 0.1615 - accuracy: 0.9466 - val_loss: 0.4366 - val_ac